# Implementing BFS and DFS for Pacman

## CS5368 Intelligent Systems - Assignment 1 Tutorial

This notebook will guide you through implementing Breadth-First Search (BFS) and Depth-First Search (DFS) algorithms for the Pacman assignment.

### Learning Objectives
- Understand the difference between BFS and DFS
- Implement graph search versions of both algorithms
- Debug common issues in search implementations
- Optimize your implementation for the autograder

## Setup and Imports

First, let's import the necessary modules and set up our environment.

In [ ]:
import sys
import os
sys.path.append('../src')

from collections import deque
from typing import List, Tuple, Set, Optional
import matplotlib.pyplot as plt
import numpy as np

# For visualization
%matplotlib inline

## Understanding the Search Problem

In Pacman, a search problem consists of:
1. **State Space**: All possible positions Pacman can be in
2. **Initial State**: Pacman's starting position
3. **Goal Test**: Check if Pacman reached the food
4. **Successor Function**: Legal moves from current position
5. **Path Cost**: Cost of sequence of actions

In [ ]:
class SearchProblem:
    """
    This class outlines the structure of a search problem.
    You don't need to change anything here.
    """
    
    def getStartState(self):
        """Returns the start state for the search problem"""
        raise NotImplementedError()
    
    def isGoalState(self, state):
        """Returns True if and only if the state is a valid goal state"""
        raise NotImplementedError()
    
    def getSuccessors(self, state):
        """
        For a given state, returns:
        successor: a list of triples (successor, action, stepCost)
        """
        raise NotImplementedError()
    
    def getCostOfActions(self, actions):
        """Returns the total cost of a particular sequence of actions"""
        raise NotImplementedError()

## Simple Test Problem

Let's create a simple grid world to test our algorithms before moving to Pacman.

In [ ]:
class SimpleGridProblem(SearchProblem):
    """
    A simple grid world for testing search algorithms.
    'S' = Start, 'G' = Goal, '#' = Wall, ' ' = Empty
    """
    
    def __init__(self, grid_string):
        lines = grid_string.strip().split('\n')
        self.grid = [list(line) for line in lines]
        self.height = len(self.grid)
        self.width = len(self.grid[0]) if self.height > 0 else 0
        
        # Find start and goal
        for i in range(self.height):
            for j in range(self.width):
                if self.grid[i][j] == 'S':
                    self.start = (i, j)
                elif self.grid[i][j] == 'G':
                    self.goal = (i, j)
    
    def getStartState(self):
        return self.start
    
    def isGoalState(self, state):
        return state == self.goal
    
    def getSuccessors(self, state):
        successors = []
        x, y = state
        
        # Check all four directions: North, South, East, West
        for dx, dy, action in [(-1,0,'North'), (1,0,'South'), (0,1,'East'), (0,-1,'West')]:
            next_x, next_y = x + dx, y + dy
            
            # Check if the next position is valid
            if (0 <= next_x < self.height and 
                0 <= next_y < self.width and 
                self.grid[next_x][next_y] != '#'):
                
                next_state = (next_x, next_y)
                cost = 1  # Uniform cost for this simple problem
                successors.append((next_state, action, cost))
        
        return successors
    
    def visualize(self, path=None):
        """Visualize the grid and optionally a path"""
        display = [row[:] for row in self.grid]
        
        if path:
            current = self.start
            for action in path:
                x, y = current
                if display[x][y] not in ['S', 'G']:
                    display[x][y] = '.'
                
                # Move to next position
                for next_state, next_action, _ in self.getSuccessors(current):
                    if next_action == action:
                        current = next_state
                        break
        
        for row in display:
            print(''.join(row))

# Create a test maze
test_maze = """
###########
#S        #
# ####### #
#         #
# ####### #
#         #
# #########
#        G#
###########
"""

problem = SimpleGridProblem(test_maze)
problem.visualize()
print(f"\nStart: {problem.getStartState()}")
print(f"Goal: {problem.goal}")

## Implementing Depth-First Search (DFS)

DFS explores as far as possible along each branch before backtracking.

### Key Points:
- Uses a **stack** (LIFO) for the frontier
- Not guaranteed to find the shortest path
- Memory efficient: O(bm) where b is branching factor, m is max depth
- Can get stuck in infinite loops without cycle detection

In [ ]:
def depthFirstSearch(problem):
    """
    Search the deepest nodes in the search tree first.
    Returns a list of actions that reaches the goal.
    """
    # Initialize the frontier with the start state
    # Each node is a tuple: (state, actions_to_reach_state)
    start_state = problem.getStartState()
    
    # Check if start is goal
    if problem.isGoalState(start_state):
        return []
    
    # Use a stack for DFS (list in Python)
    frontier = [(start_state, [])]
    
    # Keep track of explored states to avoid cycles
    explored = set()
    
    # Statistics for learning
    nodes_expanded = 0
    
    while frontier:
        # Pop from stack (LIFO)
        state, actions = frontier.pop()
        
        # Skip if already explored
        if state in explored:
            continue
        
        # Mark as explored
        explored.add(state)
        nodes_expanded += 1
        
        # Expand the state
        for successor, action, cost in problem.getSuccessors(state):
            if successor not in explored:
                # Check if we reached the goal
                if problem.isGoalState(successor):
                    print(f"DFS: Found goal! Nodes expanded: {nodes_expanded}")
                    return actions + [action]
                
                # Add to frontier
                frontier.append((successor, actions + [action]))
    
    print(f"DFS: No solution found. Nodes expanded: {nodes_expanded}")
    return []  # No solution found

# Test DFS
dfs_path = depthFirstSearch(problem)
print(f"DFS Path: {dfs_path}")
print(f"Path length: {len(dfs_path)}")
print("\nVisualization:")
problem.visualize(dfs_path)

## Implementing Breadth-First Search (BFS)

BFS explores all nodes at depth d before exploring nodes at depth d+1.

### Key Points:
- Uses a **queue** (FIFO) for the frontier
- Guaranteed to find the shortest path (in terms of number of actions)
- Memory intensive: O(b^d) where d is depth of solution
- Complete for finite state spaces

In [ ]:
def breadthFirstSearch(problem):
    """
    Search the shallowest nodes in the search tree first.
    Returns a list of actions that reaches the goal.
    """
    # Initialize
    start_state = problem.getStartState()
    
    # Check if start is goal
    if problem.isGoalState(start_state):
        return []
    
    # Use a queue for BFS
    frontier = deque([(start_state, [])])
    
    # Keep track of explored states
    explored = set([start_state])
    
    # Statistics
    nodes_expanded = 0
    max_frontier_size = 1
    
    while frontier:
        # Track max frontier size
        max_frontier_size = max(max_frontier_size, len(frontier))
        
        # Pop from queue (FIFO)
        state, actions = frontier.popleft()
        nodes_expanded += 1
        
        # Expand the state
        for successor, action, cost in problem.getSuccessors(state):
            if successor not in explored:
                # Check if we reached the goal
                if problem.isGoalState(successor):
                    print(f"BFS: Found goal! Nodes expanded: {nodes_expanded}")
                    print(f"Max frontier size: {max_frontier_size}")
                    return actions + [action]
                
                # Mark as explored when added to frontier (graph search)
                explored.add(successor)
                frontier.append((successor, actions + [action]))
    
    print(f"BFS: No solution found. Nodes expanded: {nodes_expanded}")
    return []  # No solution found

# Test BFS
bfs_path = breadthFirstSearch(problem)
print(f"BFS Path: {bfs_path}")
print(f"Path length: {len(bfs_path)}")
print("\nVisualization:")
problem.visualize(bfs_path)

## Comparing BFS and DFS

Let's compare the performance and paths found by both algorithms.

In [ ]:
def compare_algorithms(problem):
    """Compare BFS and DFS on the same problem"""
    
    print("="*50)
    print("Algorithm Comparison")
    print("="*50)
    
    # Run DFS
    print("\nDepth-First Search:")
    dfs_path = depthFirstSearch(problem)
    print(f"Path length: {len(dfs_path)}")
    print(f"Path: {dfs_path[:10]}..." if len(dfs_path) > 10 else f"Path: {dfs_path}")
    
    # Run BFS
    print("\nBreadth-First Search:")
    bfs_path = breadthFirstSearch(problem)
    print(f"Path length: {len(bfs_path)}")
    print(f"Path: {bfs_path[:10]}..." if len(bfs_path) > 10 else f"Path: {bfs_path}")
    
    print("\n" + "="*50)
    print("Key Observations:")
    print(f"- BFS found {'the same' if len(bfs_path) == len(dfs_path) else 'a different'} length path")
    print(f"- BFS path is {'optimal' if len(bfs_path) <= len(dfs_path) else 'suboptimal'} (guaranteed shortest)")
    print(f"- DFS path is {'optimal' if len(dfs_path) == len(bfs_path) else 'suboptimal'} (not guaranteed)")
    print("="*50)

compare_algorithms(problem)

## Common Pitfalls and Debugging

Here are common issues students face and how to debug them:

In [ ]:
def debug_search_algorithm(problem, algorithm_name, search_function):
    """
    Debug version of search with detailed output
    """
    print(f"\nDebugging {algorithm_name}")
    print("="*40)
    
    start_state = problem.getStartState()
    print(f"Start state: {start_state}")
    print(f"Is start goal? {problem.isGoalState(start_state)}")
    
    # Get successors of start
    successors = problem.getSuccessors(start_state)
    print(f"\nSuccessors of start state:")
    for succ, action, cost in successors:
        print(f"  {action}: {succ} (cost: {cost})")
    
    # Run the algorithm
    print(f"\nRunning {algorithm_name}...")
    path = search_function(problem)
    
    if path:
        print(f"\nSolution found with {len(path)} actions")
        
        # Verify the path
        current = start_state
        print("\nVerifying path:")
        print(f"Start: {current}")
        
        for i, action in enumerate(path):
            # Find the successor for this action
            found = False
            for succ, act, _ in problem.getSuccessors(current):
                if act == action:
                    current = succ
                    found = True
                    print(f"Step {i+1}: {action} -> {current}")
                    break
            
            if not found:
                print(f"ERROR: Invalid action '{action}' from state {current}")
                return False
        
        if problem.isGoalState(current):
            print(f"✓ Path correctly reaches goal!")
            return True
        else:
            print(f"✗ Path does not reach goal. Final state: {current}")
            return False
    else:
        print("No solution found")
        return False

# Debug both algorithms
debug_search_algorithm(problem, "DFS", depthFirstSearch)
debug_search_algorithm(problem, "BFS", breadthFirstSearch)

## Optimizations for Pacman

Here are some optimizations specific to the Pacman implementation:

In [ ]:
class PacmanSearchTips:
    """
    Tips and tricks for implementing search in Pacman
    """
    
    @staticmethod
    def tip_1_data_structures():
        """
        Use the provided data structures from util.py
        """
        print("TIP 1: Data Structures")
        print("-" * 30)
        print("For DFS: use util.Stack()")
        print("  - stack.push(item)")
        print("  - stack.pop()")
        print("  - stack.isEmpty()")
        print()
        print("For BFS: use util.Queue()")
        print("  - queue.push(item)")
        print("  - queue.pop()")
        print("  - queue.isEmpty()")
        print()
        print("For UCS: use util.PriorityQueue()")
        print("  - pq.push(item, priority)")
        print("  - pq.pop()")
        print("  - pq.isEmpty()")
    
    @staticmethod
    def tip_2_state_representation():
        """
        Understand state representation
        """
        print("TIP 2: State Representation")
        print("-" * 30)
        print("States are usually (x, y) tuples")
        print("Actions are strings: 'North', 'South', 'East', 'West', 'Stop'")
        print("getSuccessors returns: [(state, action, cost), ...]")
        print()
        print("Example:")
        print("  state = (5, 5)")
        print("  successors = problem.getSuccessors(state)")
        print("  # Returns: [((5,6), 'North', 1), ((5,4), 'South', 1), ...]")
    
    @staticmethod
    def tip_3_graph_vs_tree_search():
        """
        Implement graph search, not tree search
        """
        print("TIP 3: Graph Search vs Tree Search")
        print("-" * 30)
        print("ALWAYS use graph search (with explored set)")
        print("Tree search will revisit states and potentially loop forever")
        print()
        print("Graph search pattern:")
        print("  explored = set()")
        print("  while frontier:")
        print("      state = frontier.pop()")
        print("      if state in explored: continue")
        print("      explored.add(state)")
        print("      # ... expand state ...")
    
    @staticmethod
    def tip_4_return_format():
        """
        Return the correct format
        """
        print("TIP 4: Return Format")
        print("-" * 30)
        print("Return a list of ACTIONS, not states")
        print("Example: ['South', 'South', 'West', 'West']")
        print()
        print("NOT: [(5,5), (5,4), (5,3), (4,3)]")
        print()
        print("Store path as you search:")
        print("  frontier.push((state, path_to_state))")
        print("  # When goal found: return path_to_state + [action_to_goal]")
    
    @staticmethod
    def show_all_tips():
        tips = [
            PacmanSearchTips.tip_1_data_structures,
            PacmanSearchTips.tip_2_state_representation,
            PacmanSearchTips.tip_3_graph_vs_tree_search,
            PacmanSearchTips.tip_4_return_format
        ]
        
        for i, tip in enumerate(tips, 1):
            print("\n" + "="*40)
            tip()
        print("\n" + "="*40)

# Show all tips
PacmanSearchTips.show_all_tips()

## Testing Your Implementation

Here's how to test your implementation thoroughly:

In [ ]:
def create_test_cases():
    """Create various test cases for your algorithms"""
    
    test_cases = []
    
    # Test 1: Simple path
    simple = """
#####
#S G#
#####
"""
    test_cases.append(("Simple", simple, 2))
    
    # Test 2: Single solution
    corridor = """
#######
#S    #
##### #
#   G #
#######
"""
    test_cases.append(("Corridor", corridor, 6))
    
    # Test 3: Multiple paths
    multi_path = """
#######
#S    #
# ### #
#     #
# ### #
#    G#
#######
"""
    test_cases.append(("Multi-path", multi_path, 8))
    
    return test_cases

def run_test_suite():
    """Run comprehensive tests"""
    test_cases = create_test_cases()
    
    print("Running Test Suite")
    print("="*50)
    
    for name, maze, expected_bfs_length in test_cases:
        print(f"\nTest: {name}")
        print("-"*30)
        
        prob = SimpleGridProblem(maze)
        
        # Test BFS
        bfs_result = breadthFirstSearch(prob)
        bfs_pass = len(bfs_result) == expected_bfs_length
        print(f"BFS: {'✓ PASS' if bfs_pass else '✗ FAIL'} (length: {len(bfs_result)}, expected: {expected_bfs_length})")
        
        # Test DFS
        dfs_result = depthFirstSearch(prob)
        dfs_pass = len(dfs_result) > 0  # Just check if solution found
        print(f"DFS: {'✓ PASS' if dfs_pass else '✗ FAIL'} (length: {len(dfs_result)})")
        
        # Show if DFS found optimal
        if len(dfs_result) == expected_bfs_length:
            print("  → DFS found optimal path!")
        else:
            print(f"  → DFS path is {len(dfs_result) - expected_bfs_length} steps longer")

run_test_suite()

## Summary and Next Steps

### What You've Learned:
1. ✅ How to implement DFS using a stack
2. ✅ How to implement BFS using a queue
3. ✅ The importance of cycle detection (explored set)
4. ✅ How to debug search algorithms
5. ✅ The trade-offs between BFS and DFS

### Next Steps for Assignment 1:

1. **Implement in `search.py`:**
   - Copy your `depthFirstSearch` function
   - Copy your `breadthFirstSearch` function
   - Use `util.Stack()` and `util.Queue()` instead of Python's built-in structures

2. **Test with Pacman:**
   ```bash
   python pacman.py -l tinyMaze -p SearchAgent -a fn=dfs
   python pacman.py -l mediumMaze -p SearchAgent -a fn=bfs
   ```

3. **Check autograder:**
   ```bash
   python autograder.py -q q1  # For DFS
   python autograder.py -q q2  # For BFS
   ```

### Common Autograder Issues:

- **"Stack overflow"**: You have infinite recursion or aren't checking explored states
- **"Wrong action sequence"**: You're returning states instead of actions
- **"Takes too long"**: You're doing tree search instead of graph search
- **"Wrong path length"**: For BFS, ensure you're not exploring the same state twice

### Good luck with your implementation! 🎮👻